In [1]:
import findspark

findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import col,isnan, when, count
import pyspark.sql.functions as F

from configparser import ConfigParser

config = ConfigParser()
# create your own config.ini in root of project folder to store project configurations
config.read('config.ini')

pathfile = config.get('main', 'dirty_csv')  

spark = SparkSession.builder \
    .config("spark.driver.memory", "15g") \
    .appName("SparkFlight").getOrCreate()



In [2]:
# read preprocessed data (15mil rows)
preproc_data = spark.read.csv('preprocessed_data.csv', inferSchema='true', header='true', mode='PERMISSIVE', encoding='ISO-8859-1')

In [3]:
from math import factorial
from itertools import combinations, product

def generate_deps(lhs_columns, rhs_columns, lhsSize: int, alreadyFound = []):
    # generate left hand sides
    lhss = combinations(lhs_columns, r=lhsSize)
    # add right hand sides
    deps = product(lhss, rhs_columns)
    # convert to list format, where ['a', 'b', 'c'] corresponds to 'a', 'b' -> 'c'
    deps = map(lambda dep: list(dep[0]) + [dep[1]], deps)

    # remove tirivial dependencies (where RHS in LHS)
    deps = filter(lambda dep: not dep[-1] in dep[:-1], deps)

    # used to test if dependency is implied by some found dependency
    includes_lhs = lambda found_dep, dep: all(attr in dep[:-1] for attr in found_dep[:-1])
    implied = lambda dep: any(map(lambda found_dep: found_dep[-1] == dep[-1] and includes_lhs(found_dep, dep), alreadyFound))
    # remove already found dependencies
    deps = filter(lambda dep: not implied(dep), deps)

    return list(deps)   

def gen_contin_cells(x, bv_candidate_FDs, lhs_size):
    combs = []
    candidate_FDs = [tuple(candidate_FD) for candidate_FD in bv_candidate_FDs.value]
#     for comb in candidate_FDs:
    for i, fd in enumerate(candidate_FDs):
# rows in form: (((lhs, rhs), (value(s) lhs, value rhs)), count)
        combs.append(((i, tuple(x[column] for column in fd)),1))
    return combs

def calc_combinations(x):
#     combinations of 2
    return factorial(x)/(factorial(2)*factorial(x-2)) if x >= 2 else 0

def map_value_to_combs_count(rdd_row):
    lhs_plus_rhs = rdd_row[0][0]
#     value of rhs is always last item. Strip it to only depend on value of lhs in key
    values_lhs_tup = rdd_row[0][1][:-1]
    reduced_count_rows = rdd_row[1]
    return ((lhs_plus_rhs, values_lhs_tup), calc_combinations(reduced_count_rows))
        
def calc_percentage(x, y):
#     normalize to percentage value in [0, 1]
    result = y/x if x >= y else x/y
#     if normalized percentage value = 1 then set to 1.5 instead so this value cannot be confused
#     with a possible total 2 comb count of 1 for lhs
#     as total combination count always has to be integer
    result = 1.5 if result == 1.0 else result 
    return result

def map_total_comb_to_zero_perc(rdd_row):
    lhs_plus_rhs = rdd_row[0][0] 
    value = rdd_row[1]
#     rdd_row could be row with total 2 comb count per lhs as value that did not get normalized to [0, 1] range as percentage
#     as all rows for that lhs had unique rhs values so there was no row with possible combinations of 2 equal lhs + rhs per lhs
#     to reduce with. Rows with unique lhs + rhs were filtered in step #3. 
#     lhs that has no possible combinations of 2 equal lhs + rhs per lhs will be set to 0%
#     otherwise rdd_row has normalized percentage as value but in the case of percentage being 100% value = 1.5 instead of 1.
    if value >= 1 and value != 1.5:
        value = 0
    elif value == 1.5: # set 100%'s back to 1
        value = 1
    return (lhs_plus_rhs, value)
    
def rem_rhs_value_from_key(rdd_row):
    lhs_plus_rhs = rdd_row[0][0]
#     value of rhs is always last item. Strip it to only depend on value of lhs in key
    values_lhs_tup = rdd_row[0][1][:-1]
    value = rdd_row[1]
    return ((lhs_plus_rhs, values_lhs_tup), value)
    
        

In [4]:
# #  get a sample. This is just for testing locally. Should be whole preprocessed dataset.
# len_sample = 16_000
# len_preproc_data = 15_000_000
# cont_sample_data = preproc_data.sample(True, fraction=len_sample/len_preproc_data).cache()

In [ ]:
import time

max_lhs = 3
col_names = preproc_data.columns
found_soft_dep = []
perc_threshold = 0.7

for i in range(1, max_lhs+1):
    tic = time.perf_counter()
    candidate_FDs = generate_deps(col_names, col_names, i, found_soft_dep)
#     broadcast the candidate fds only once to all nodes: http://spark.apache.org/docs/latest/rdd-programming-guide.html#broadcast-variables
    broadcast_candidate_FDs = spark.sparkContext.broadcast(candidate_FDs)
    
    print(f"Running full dataset over {len(candidate_FDs)} possible Soft Dependencies with threshold: {perc_threshold} and lhs: {i}")
# #    sample only for local use to test
#     flat_columns = cont_sample_data.rdd.flatMap(lambda x: gen_contin_cells(x, broadcast_candidate_FDs, lhs_size=i)) # 1

#     create all column combs per row
    flat_columns = preproc_data.rdd.flatMap(lambda x: gen_contin_cells(x, broadcast_candidate_FDs, lhs_size=i)) # 1
#     cache or not?? only used one time extra this rdd later? check if this actually wins time
    c_flat_columns = flat_columns.reduceByKey(lambda x,y: x+y).cache() # 2
#     unique lhs + rhs rows are not needed to calculate possible combinations of 2 rows with equal lhs.
    f_c_flat_columns = c_flat_columns.filter(lambda x: x[1] >= 2) # 3
#     map identical lhs + rhs occurences to possible combs of 2
    calc_combs = f_c_flat_columns.map(lambda x: map_value_to_combs_count(x)) # 4
#     reduce amount of possible combinations of 2 equal lhs + rhs per lhs
    reduce_combs_by_lhs = calc_combs.reduceByKey(lambda x,y: x+y) # 5
    
#     make use of already cached reduced rdd from step #2
#     make sure to only reduce on lhs as we want total 2 comb count for lhs
    row_c_for_lhs = c_flat_columns.map(lambda x: rem_rhs_value_from_key(x))
    red_c_for_lhs = row_c_for_lhs.reduceByKey(lambda x,y: x+y)
#     Filter out rows with unique lhs as they cannot match with another equal lhs row
    filt_red_c_for_lhs = red_c_for_lhs.filter(lambda x: x[1] >= 2)
#     calculate total number of 2 row combs per lhs
    map_total_combs_lhs = filt_red_c_for_lhs.mapValues(lambda value: calc_combinations(value))
    
#     now union the per (lhs, rhs) comb count rdd and per (lhs) comb count rdd
    total_and_eq_combs = map_total_combs_lhs.union(reduce_combs_by_lhs)# 8
#     calc percentage per lhs per FD
#     this could be bottleneck as now every key only has 2 rows so low chance of being able to combine locally?
    percentage_per_lhs = total_and_eq_combs.reduceByKey(lambda x,y: calc_percentage(x,y)) # 9
    
    mapped_percentages = percentage_per_lhs.map(lambda x: map_total_comb_to_zero_perc(x))
#     reduce by value (percentage) per lhs. Keep separate count to calculate the mean over the percentages of one FD combination (mapValues).
    means_percentages = mapped_percentages.mapValues(lambda value: (value, 1)) \
                                            .reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])) \
                                            .mapValues(lambda value: value[0]/value[1])
    
    filter_threshold_fds = means_percentages.filter(lambda x: x[1] >= perc_threshold)
#     take only fd index and map index to full fd
    take_only_fd_indic = filter_threshold_fds.map(lambda x: broadcast_candidate_FDs.value[x[0]])
    soft_fd_comb_list = take_only_fd_indic.collect()
#     form: [["lhs column", "lhs column", "rhs column"], ....] depending on value of i in loop for amount of lhs columns
    found_soft_dep += soft_fd_comb_list
    
    toc = time.perf_counter()
    print(f"discovering soft dependencies took {toc - tic:0.4f} seconds")
    print(f'amount of dependencies found: {len(found_soft_dep)}')
    print('found soft dependencies:')
    print(found_soft_dep)
    



Running full dataset over 342 possible Soft Dependencies with threshold: 0.7 and lhs: 1


# These cells are just for testing locally


In [ ]:
col_names = preproc_data.columns
comb_dict = gen_col_combs(col_names,3)
len(comb_dict['lhs_1'])

In [ ]:
found_combs = [('TailNum', 'SecurityDelay'), ('CRSArrTimestamp', 'NASAndWeatherDelay'), ('Distance', 'SecurityDelay'), ('UniqueCarrier', 'LateAircraftAndCarrierDelay'), ('AirTime', 'SecurityDelay'), ('Origin', 'NASAndWeatherDelay'), ('Origin', 'SecurityDelay'), ('ArrTimestamp', 'LateAircraftAndCarrierDelay'), ('ActualElapsedTime', 'NASAndWeatherDelay'), ('Dest', 'SecurityDelay'), ('CRSDepTimeStamp', 'LateAircraftAndCarrierDelay'), ('LateAircraftAndCarrierDelay', 'SecurityDelay'), ('ArrTimestamp', 'SecurityDelay'), ('Distance', 'NASAndWeatherDelay'), ('DepTimestamp', 'NASAndWeatherDelay'), ('ActualElapsedTime', 'SecurityDelay'), ('Dest', 'NASAndWeatherDelay'), ('CRSElapsedTime', 'SecurityDelay'), ('TaxiOut', 'LateAircraftAndCarrierDelay'), ('ArrTimestamp', 'NASAndWeatherDelay'), ('CRSArrTimestamp', 'SecurityDelay'), ('UniqueCarrier', 'SecurityDelay'), ('Distance', 'LateAircraftAndCarrierDelay'), ('TaxiIn', 'LateAircraftAndCarrierDelay'), ('TaxiOut', 'SecurityDelay'), ('Dest', 'LateAircraftAndCarrierDelay'), ('CRSArrTimestamp', 'LateAircraftAndCarrierDelay'), ('TailNum', 'NASAndWeatherDelay'), ('ArrDelay', 'SecurityDelay'), ('NASAndWeatherDelay', 'SecurityDelay'), ('TaxiIn', 'SecurityDelay'), ('DepTimestamp', 'LateAircraftAndCarrierDelay'), ('UniqueCarrier', 'NASAndWeatherDelay'), ('AirTime', 'NASAndWeatherDelay'), ('CRSDepTimeStamp', 'NASAndWeatherDelay'), ('Origin', 'LateAircraftAndCarrierDelay'), ('TailNum', 'UniqueCarrier'), ('CRSElapsedTime', 'NASAndWeatherDelay'), ('ActualElapsedTime', 'LateAircraftAndCarrierDelay'), ('CRSElapsedTime', 'LateAircraftAndCarrierDelay'), ('DepTimestamp', 'SecurityDelay'), ('DepDelay', 'SecurityDelay'), ('TailNum', 'LateAircraftAndCarrierDelay'), ('AirTime', 'LateAircraftAndCarrierDelay'), ('CRSDepTimeStamp', 'SecurityDelay')]

In [ ]:
# comb_list_lhs1 = generate_deps(col_names, col_names, 1, [])
# print(len(comb_list_lhs1))
# comb_list_lhs1

In [ ]:
comb_list_lhs2 = generate_deps(col_names, col_names, 2, [])
print(len(comb_list_lhs2))
comb_list_lhs2

In [ ]:
comb_list_lhs2_min_found_with_lhs1 = generate_deps(col_names, col_names, 2, found_combs)
# comb_list_lhs2_min_found_with_lhs1 = generate_deps(col_names, col_names, 2, [])
# comb_list_lhs2_min_found_with_lhs1 = generate_deps(col_names, col_names, 1, found_combs)
print(len(comb_list_lhs2_min_found_with_lhs1))
comb_list_lhs2_min_found_with_lhs1

In [ ]:
found_combs_transf